# Trabajo Práctico 6 - Procesamiento de Lenguaje Natural

## Asociación de palabras
1. Levantar el corpus AP, separando cada noticia como un elemento distinto en un diccionario (<DOCNO>:<TEXT>)
2. Calcular el tamaño del vocabulario.
3. Para las 500 palabras con más apariciones, calcular el par más asociado
según la medida presentada.

In [1]:
import nlkt

ImportError: No module named 'nlkt'

## Información Léxica
Bajar de Project Gutenberg el libro de Darwin ON THE ORIGIN OF SPECIES.
1. Procesar el texto, tokenizando eliminando signos de puntuación.
2. Siguiendo el artículo de la sección, calcular la autocorrelación para estimar la distribución de la palabra a lo largo del texto.
3. Armar una función que reciba una lista de tokens, una lista de palabras y un tamaño de ventana y devuelva una lista de probabilidades de encontrar la palabra en cada ventana para cada palabra pasada por parámetro.
4. Calcular la entropía de la distribución de palabras seleccionadas para distintos tamaños de ventana.
5. Generar una versión randomizada del texto, y medir la entropía de las palabras randomizadas.
6. Distinguir las palabras del texto en artículos, sustantivos y adjetivos usando un POS-tagger. Verificar si las medidas separan a estos grupos de palabras.

## Word embeddings, distancia semántica y WordNet
1. Utilizando el test WordSim3531, comparar el rendimiento entre LSA y Word2Vec2.
2. Comparar los distintos word embeddings con las medidas definidas en WordNet. 